In [ ]:
import sys
from pathlib import Path
from google.colab import drive

GIT_URL = "https://github.com/EtsuKeks/financial_models.git"
PROJECT_DIR = Path("/content/financial_models")
REMOTE_NAME = "gdrive"
DVC_URL = "gdrive://1V41rlDHL7BvMwoc72ZBLAxfXTcnBd2Hk"
# Secrets file pydrive_cred.json generated locally with generate_pydrive_cred.py
PDRIVE_CRED_DRIVE = Path("/content/drive/MyDrive/financial_models_secrets/pydrive_cred.json")

# GIT clone/pull from GIT_URL
PROJECT_DIR.mkdir(parents=True, exist_ok=True)
PROJECT_DIR_STR = str(PROJECT_DIR)
!bash -lc 'if [ -d "{PROJECT_DIR_STR}/.git" ]; then cd "{PROJECT_DIR_STR}" && git fetch --all --prune && git checkout master && git pull --ff-only; else git clone --branch master "{GIT_URL}" "{PROJECT_DIR_STR}"; fi'

# Install dependencies
!cd {PROJECT_DIR_STR} && python -m pip install -r requirements.txt

# Prepare pydrive cached credentials and configure DVC remote to use OAuth client
drive.mount("/content/drive", force_remount=False)
!cd {PROJECT_DIR_STR} && dvc remote add -d {REMOTE_NAME} {DVC_URL} --force
!cd {PROJECT_DIR_STR} && dvc remote modify {REMOTE_NAME} gdrive_user_credentials_file {str(PDRIVE_CRED_DRIVE)} --local

# DVC pull (should use cached credentials)
!cd {PROJECT_DIR_STR} && dvc pull -r {REMOTE_NAME} -v

# Run pipeline
sys.path.insert(0, str(PROJECT_DIR))
import pipeline; from src.utils.config import settings
pipeline.run_pipeline()

# Add and push output to DVC
!cd {PROJECT_DIR_STR} && dvc add data/outputs/{settings.ppl.output_csv}
!cd {PROJECT_DIR_STR} && dvc push -r {REMOTE_NAME} -v

# Commit the generated .dvc metadata so it is available in git
!cd {PROJECT_DIR_STR} && git add data/outputs/{settings.ppl.output_csv}.dvc || true
!cd {PROJECT_DIR_STR} && git commit -m "Add DVC metadata for {settings.ppl.output_csv}" || echo "No git changes to commit"
!cd {PROJECT_DIR_STR} && git push origin master